In [7]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()

In [8]:
# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'distilbert-base-uncased',
    'HF_TASK':'text-classification'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    env=hub,
    role=role, 
)

In [9]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1, # number of instances
    instance_type='ml.m5.12xlarge' # ec2 instance type
)

-----!

In [13]:
predictor.endpoint_name

'huggingface-pytorch-inference-2022-10-04-00-07-12-419'

In [12]:
predictor.predict({
    'inputs': "I am super happy right now."
})

[{'label': 'LABEL_0', 'score': 0.5618108510971069}]

In [11]:
import boto3
import json
client = boto3.client('sagemaker-runtime')
content_type = "application/json"
request_body = {'inputs': "I am super happy right now."}
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)
response = client.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType=content_type,
    Body=payload)
result = response['Body'].read()
result

b'[{"label":"LABEL_1","score":0.5247413516044617}]'

In [ ]:
#!pip install locust

In [1]:
!which locust

/home/ec2-user/anaconda3/envs/python3/bin/locust


In [5]:
%%sh
cd locust
./distributed.sh

b'[{"label":"LABEL_1","score":0.5399844646453857}]'
b'[{"label":"LABEL_0","score":0.5618108510971069}]'
b'[{"label":"LABEL_1","score":0.5529375076293945}]'
b'[{"label":"LABEL_0","score":0.5658608078956604}]'
b'[{"label":"LABEL_1","score":0.5367456078529358}]'
b'[{"label":"LABEL_0","score":0.507613480091095}]'
b'[{"label":"LABEL_0","score":0.5099622011184692}]'
b'[{"label":"LABEL_0","score":0.521774172782898}]'
b'[{"label":"LABEL_1","score":0.5255634188652039}]'
b'[{"label":"LABEL_0","score":0.5063676834106445}]'
b'[{"label":"LABEL_1","score":0.5089067816734314}]'
b'[{"label":"LABEL_0","score":0.5106375813484192}]'
b'[{"label":"LABEL_0","score":0.5380833745002747}]'
b'[{"label":"LABEL_1","score":0.5024497509002686}]'
b'[{"label":"LABEL_0","score":0.5089787840843201}]'
b'[{"label":"LABEL_1","score":0.5373550057411194}]'
b'[{"label":"LABEL_1","score":0.530104398727417}]'
b'[{"label":"LABEL_0","score":0.5679922699928284}]'
b'[{"label":"LABEL_0","score":0.5174779891967773}]'
b'[{"label":"LA

[2022-10-04 00:47:26,746] ip-172-16-14-121.ec2.internal/WARNING/locust.main: System open file limit '8192' is below minimum setting '10000'.
It's not high enough for load testing, and the OS didn't allow locust to increase it by itself.
See https://github.com/locustio/locust/wiki/Installation#increasing-maximum-number-of-open-files-limit for more info.
[2022-10-04 00:47:26,746] ip-172-16-14-121.ec2.internal/WARNING/locust.main: System open file limit '8192' is below minimum setting '10000'.
It's not high enough for load testing, and the OS didn't allow locust to increase it by itself.
See https://github.com/locustio/locust/wiki/Installation#increasing-maximum-number-of-open-files-limit for more info.
[2022-10-04 00:47:26,747] ip-172-16-14-121.ec2.internal/WARNING/locust.main: System open file limit '8192' is below minimum setting '10000'.
It's not high enough for load testing, and the OS didn't allow locust to increase it by itself.
See https://github.com/locustio/locust/wiki/Installat

In [8]:
import pandas as pd
locust_data = pd.read_csv('locust/results_stats.csv')
for index, row in locust_data.head(n=2).iterrows():
     print(index, row)

0 Type                     InvokeEndpoint
Name                          SageMaker
Request Count                      8510
Failure Count                         0
Median Response Time                 63
Average Response Time         64.974015
Min Response Time             48.730953
Max Response Time            170.098008
Average Content Size                0.0
Requests/s                   141.840223
Failures/s                          0.0
50%                                  63
66%                                  64
75%                                  64
80%                                  65
90%                                  71
95%                                  81
98%                                  94
99%                                 100
99.9%                               150
99.99%                              170
100%                                170
Name: 0, dtype: object
1 Type                            NaN
Name                     Aggregated
Request Count        